In [97]:
import csv
import pickle
from datetime import date, datetime, timedelta
import numpy as np
import math
import copy

## Convert row ftse file data to pickle

In [98]:
# ToDo: move all methods to a separate file and import them

def convert_txt_to_dict(filename):
    content = []

    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            content.append(row)

    data_ftse_100_dict = {}
    tickers_list = [ticker[:-2] for ticker in content[0][1:]]
    data_ftse_100_dict = {}

    for row in content[1:]:
        data_ftse_100_dict[row[0]] = {}
        for i, ticker in enumerate(tickers_list):
            data_ftse_100_dict[row[0]][ticker] = row[i + 1]
    return data_ftse_100_dict

    
txt_filename = 'ftse_100_time_data.csv'
path_to_save = "./data_ftse_interp_0_1_2001_2006.p"

data_ftse_100_dict = convert_txt_to_dict(txt_filename)

# pickle.dump(data_ftse_100_dict, open(path_to_save, "wb"))

## Library to interpolate ftse dataset

How to write `add_dates` method
http://stackoverflow.com/questions/3240458/how-to-increment-the-day-in-datetime-python

In [115]:
def datetime_to_date(datetime_date):
    datetime_date.date().year
    str_date = '{}-{}-{}'.format(datetime_date.date().strftime('%Y'), datetime_date.date().strftime('%m'), datetime_date.date().strftime('%d'))
    return str_date
    
def date_to_datetime(date):
    return datetime.strptime(date, '%Y-%m-%d')

def find_start_end_dates(ticker, dates_dict):
    # this method looks when a company started to be in ftse 100. Assume that some dates may not have dictionaries
    # with ticker. No ticker in keys.
    start = ""
    end = ""
    for date, stock_values in dates_dict.items():
        if ticker in stock_values.keys():
            if stock_values[ticker]:
                # check if start is empty
                if not start:
                    start = date
                else:
                    end = date
    
    return start, end # dates in datetime format

def get_dates_range(start_date, end_date):
    # assume start_date and end_date are in datetime format
    dates_range = []
    delta = end_date - start_date
    date_index = start_date
    for i in range(delta.days + 1):
        dates_range.append(date_index)
        date_index += timedelta(days=1)
    return dates_range

def add_interpolated_values(dates_range, ticker, dates_dict):
    modified_dates_dict = dates_dict
    start_indicator = dates_range[0]
    end_indicator = dates_range[-1]
    interpolated_value = (float(modified_dates_dict[datetime_to_date(start_indicator)][ticker]) + float(modified_dates_dict[datetime_to_date(end_indicator)][ticker])) / 2
    for date in dates_range[1:-1]:
        # Initialise new date
        try:
            value = modified_dates_dict[datetime_to_date(date)]
        except KeyError:
            modified_dates_dict[datetime_to_date(date)] = {}
        
        modified_dates_dict[datetime_to_date(date)][ticker] = interpolated_value

    return modified_dates_dict # add average values using start_indicator and end_indicator
    
def add_missing_date(missing_date, complete_dates_dict, ticker):
    #print("here")
    #print(len(list(complete_dates_dict.keys())))
    try:
        dic = complete_dates_dict[missing_date]
        try:
            value = complete_dates_dict[missing_date][ticker]
        except KeyError:
            complete_dates_dict[missing_date][ticker] = ""
    except KeyError:
        complete_dates_dict[missing_date] = {}
        complete_dates_dict[missing_date][ticker] = ""

def initialize_date(date_init, all_dates_dict, ticker, initial_value):
    if not all_dates_dict[datetime_to_date(date_init)][ticker]:
        all_dates_dict[datetime_to_date(date_init)][ticker] = initial_value

def interpolate(dates_dict, ticker, start_date, end_date):
    # assume start_date and end_date are in datetime format
    dates_range = get_dates_range(start_date, end_date)
    start_indicator = dates_range[0]
    dates_dict_interp = dates_dict
    
    add_missing_date(datetime_to_date(start_indicator), dates_dict_interp, ticker)
    initialize_date(start_indicator, dates_dict_interp, ticker, 500)
    
    for date in dates_range[1:]:
        add_missing_date(datetime_to_date(date), dates_dict_interp, ticker)
        #print("start", datetime_to_date(start_indicator))
        #print("end", datetime_to_date(end_indicator))
        #print("date", datetime_to_date(date))
        #print("ticker_value", dates_dict_interp[datetime_to_date(date)][ticker])
        #print(dates_dict_interp[datetime_to_date(date)][ticker])
        if dates_dict_interp[datetime_to_date(date)][ticker]:
            #print(dates_dict_interp[datetime_to_date(date)][ticker])
            #print(datetime_to_date(date))
            #print("empty")
            if (date - start_indicator).days >= 1:
                dates_range_interp = get_dates_range(start_indicator, date)
                dates_dict_interp = add_interpolated_values(dates_range_interp, ticker, dates_dict_interp)
                start_indicator = date
            else:
                start_indicator = date
            
    return dates_dict_interp

def get_existing_companies(date_point, data_ftse_100):
    tickers_list = list(data_ftse_100['2012-07-11'].keys())
    valid_tickers = []
    for ticker in tickers_list:
        start, end = find_start_end_dates(ticker, data_ftse_100)
        if date_to_datetime(date_point) > date_to_datetime(start):
            valid_tickers.append(ticker)
    return valid_tickers

def interpolate_all(tickers_lst, data_ftse, date_init, date_end):
    data_ftse_100_dict_interp = copy.deepcopy(data_ftse) 
    for ticker in tickers_lst:
        print(ticker)
        data_ftse_100_dict_interp = interpolate(data_ftse_100_dict_interp, ticker, date_to_datetime(date_init), date_to_datetime(date_end))
    
    return data_ftse_100_dict_interp

In [116]:
start, end = find_start_end_dates("III", data_ftse_100_dict)
print(start, end)

1993-03-31 2015-04-29


In [117]:
valid_tickers = get_existing_companies("2000-04-29", data_ftse_100_dict)

print(len(valid_tickers))
print(valid_tickers)

76
['AAL', 'ABF', 'ADN', 'AGK', 'AHT', 'ANTO', 'ARM', 'AV', 'AZN', 'BA', 'BAB', 'BARC', 'BATS', 'BDEV', 'BG', 'BLND', 'BLT', 'BNZL', 'BP', 'BT-A', 'CNA', 'CPG', 'CPI', 'CRH', 'DGE', 'GKN', 'GSK', 'HMSO', 'HSBA', 'III', 'IMT', 'INTU', 'ITV', 'JMAT', 'KGF', 'LAND', 'LGEN', 'LLOY', 'MGGT', 'MKS', 'MRW', 'NG', 'NXT', 'OML', 'PRU', 'PSN', 'PSON', 'RB', 'RBS', 'REL', 'RIO', 'RR', 'RRS', 'RSA', 'SAB', 'SBRY', 'SDR', 'SGE', 'SHP', 'SKY', 'SMIN', 'SN', 'SSE', 'STAN', 'STJ', 'SVT', 'TPK', 'TSCO', 'TW', 'ULVR', 'UU', 'VOD', 'WEIR', 'WOS', 'WPP', 'WTB']


In [118]:

# first: data_ftse_interp = interpolate_all(valid_tickers[:1], data_ftse_100_dict, '2001-07-13', '2006-07-11')
data_ftse_interp = interpolate_all(valid_tickers[0:10], data_ftse_100_dict, '2010-07-14', '2015-04-11')

#pickle.dump(data_ftse_interp, open("data_ftse_interp_0_1_2001_2006.p", "wb" ))

AAL
ABF
ADN
AGK
AHT
ANTO
ARM
AV
AZN
BA


In [119]:
# pickle.dump(data_ftse_interp, open("data_ftse_interp_0_60_2000_2013.p", "wb" ))
#data_ftse_interp = pickle.load(open("data_ftse_interp_0_60_2000_2013.p", "rb"))

In [120]:
dat_range = get_dates_range(date_to_datetime('2011-07-14'), date_to_datetime('2014-07-11'))

for dat in dat_range:
    # print("ABF", data_ftse_interp[datetime_to_date(dat)]["ABF"], datetime_to_date(dat))
    print("good", data_ftse_interp[datetime_to_date(dat)]["AAL"])
    try:
        print(datetime_to_date(dat), data_ftse_100_dict[datetime_to_date(dat)]["AAL"])
    except:
        pass
    

good 2999.0
2011-07-14 2999.0
good 2975.0
2011-07-15 2975.0
good 2963.0
good 2963.0
good 2951.0
2011-07-18 2951.0
good 2929.5
2011-07-19 2929.5
good 2987.5
2011-07-20 2987.5
good 2988.5
2011-07-21 2988.5
good 3011.0
2011-07-22 3011.0
good 2995.5
good 2995.5
good 2980.0
2011-07-25 2980.0
good 3050.0
2011-07-26 3050.0
good 3046.5
2011-07-27 3046.5
good 2981.0
2011-07-28 2981.0
good 2910.0
2011-07-29 2910.0
good 2928.5
good 2928.5
good 2947.0
2011-08-01 2947.0
good 2855.0
2011-08-02 2855.0
good 2750.0
2011-08-03 2750.0
good 2777.0
2011-08-04 2777.0
good 2540.0
2011-08-05 2540.0
good 2505.0
good 2505.0
good 2470.0
2011-08-08 2470.0
good 2369.0
2011-08-09 2369.0
good 2498.0
2011-08-10 2498.0
good 2415.0
2011-08-11 2415.0
good 2398.0
2011-08-12 2398.0
good 2434.75
good 2434.75
good 2471.5
2011-08-15 2471.5
good 2434.0
2011-08-16 2434.0
good 2423.0
2011-08-17 2423.0
good 2426.0
2011-08-18 2426.0
good 2287.5
2011-08-19 2287.5
good 2257.0
good 2257.0
good 2226.5
2011-08-22 2226.5
good 2278.5
20

2013-04-26 1642.5
good 1629.75
good 1629.75
good 1617.0
2013-04-29 1617.0
good 1606.0
2013-04-30 1606.0
good 1576.0
2013-05-01 1576.0
good 1559.5
2013-05-02 1559.5
good 1576.0
2013-05-03 1576.0
good 1605.75
good 1605.75
good 1635.5
2013-05-06 1635.5
good 1638.0
2013-05-07 1638.0
good 1630.5
2013-05-08 1630.5
good 1668.0
2013-05-09 1668.0
good 1670.5
2013-05-10 1670.5
good 1633.75
good 1633.75
good 1597.0
2013-05-13 1597.0
good 1585.0
2013-05-14 1585.0
good 1556.5
2013-05-15 1556.5
good 1550.0
2013-05-16 1550.0
good 1551.0
2013-05-17 1551.0
good 1558.25
good 1558.25
good 1565.5
2013-05-20 1565.5
good 1556.5
2013-05-21 1556.5
good 1624.0
2013-05-22 1624.0
good 1565.0
2013-05-23 1565.0
good 1575.0
2013-05-24 1575.0
good 1565.75
good 1565.75
good 1556.5
2013-05-27 1556.5
good 1553.0
2013-05-28 1553.0
good 1533.0
2013-05-29 1533.0
good 1533.5
2013-05-30 1533.5
good 1565.5
2013-05-31 1565.5
good 1545.0
good 1545.0
good 1524.5
2013-06-03 1524.5
good 1557.5
2013-06-04 1557.5
good 1500.0
2013-0

## Calculate correlations

In [122]:
def get_components_matrix(time_window, time_start, time_end, ticker_list, dates_dict):
    components_distance = {}
    dates_range = get_dates_range(date_to_datetime(time_start), date_to_datetime(time_end))
    
    for date in dates_range:
        vectors = []
        for ticker in ticker_list:
            vectors.append(get_s_vector(time_window, ticker, dates_dict, date))
        components_distance[datetime_to_date(date)] = np.corrcoef(vectors)
    return components_distance # dict where each key is time and value is distance

def get_s_vector(time_window, ticker, dates_dict, initial_time):
    s_vector = []
    end_time = initial_time + timedelta(days=time_window)
    dates_window = get_dates_range(initial_time, end_time)
    for date in dates_window:
        s_vector.append(get_s(date, ticker, dates_dict))
    return s_vector

def get_s(time, ticker, dates_dict):
    s = np.log(float(dates_dict[datetime_to_date(time + timedelta(days=1))][ticker])) - np.log(float(dates_dict[datetime_to_date(time)][ticker]))
    return s

ticker_1 = valid_tickers[5:][0]
ticker_2 = valid_tickers[5:7][1]  
#ticker_1 = "ARM"
#ticker_2 = "ANTO" 
#print(np.log(float(data_ftse_100_dict_interp[datetime_to_date(date_to_datetime('2013-04-01') + timedelta(days=1))][ticker])))
dist = get_components_matrix(45, '2012-07-13', '2013-07-15', valid_tickers[0:4], data_ftse_interp)
print(dist)
#numpy.corrcoef

{'2012-07-13': array([[ 1.        ,  0.0503901 ,  0.10666312, -0.00988581],
       [ 0.0503901 ,  1.        ,  0.44133805,  0.39424062],
       [ 0.10666312,  0.44133805,  1.        ,  0.2825074 ],
       [-0.00988581,  0.39424062,  0.2825074 ,  1.        ]]), '2012-07-14': array([[ 1.        ,  0.07051112,  0.13994959, -0.01636753],
       [ 0.07051112,  1.        ,  0.44972989,  0.3895652 ],
       [ 0.13994959,  0.44972989,  1.        ,  0.27320001],
       [-0.01636753,  0.3895652 ,  0.27320001,  1.        ]]), '2012-07-15': array([[ 1.        ,  0.11714953,  0.11298018, -0.02221961],
       [ 0.11714953,  1.        ,  0.41824723,  0.37420769],
       [ 0.11298018,  0.41824723,  1.        ,  0.27418167],
       [-0.02221961,  0.37420769,  0.27418167,  1.        ]]), '2012-07-16': array([[ 1.        ,  0.06825314,  0.13266329, -0.01638559],
       [ 0.06825314,  1.        ,  0.39754883,  0.36837886],
       [ 0.13266329,  0.39754883,  1.        ,  0.27515052],
       [-0.01638559,  